# ChatGPT OpenAI Economic Census & FRED Data
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1eClN6J0KEw2N3KRLSre3xA3Rh190dYJ8?usp=sharing)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | Open AI API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | String |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 13th May 2023 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

In [6]:
!pip install -q langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.3/817.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00


## <font color="blue">Imports</font>

In [7]:
from google.colab import files
from getpass import getpass
import io
import pandas as pd
import pandas_datareader as pdr
import plotly.express as px
from langchain import OpenAI
from langchain.agents import create_pandas_dataframe_agent

pd.set_option("display.max_columns", None)

In [8]:
# open ai key
openai_api_key = getpass('Enter the open ai key: ')

Enter the open ai key: ··········


## <font color="blue">Data</font>


### Get Unemployment Data

In [30]:
# transform file into pandas dataframe
df_e = pdr.DataReader(['FLHILL7URN'], 'fred', '2000-01-01', '2023-05-01')
print('Number of rows:', len(df_e))
print('Number of columns:', len(df_e.columns))
df_e.head()

Number of rows: 279
Number of columns: 1


,FLHILL7URN
DATE,
2000-01-01,3.5
2000-02-01,3.2
2000-03-01,3.1
2000-04-01,3.0
2000-05-01,3.2


In [31]:
px.line(df_e.reset_index(), x='DATE', y='FLHILL7URN')

### Get Unemployment, Median DOM, Market Hotness

In [32]:
param_dict = {
    'unemployment':'FLHILL7URN',
    'median_days_on_market':'MEDDAYONMAR12057',
    'market_hotness_ratio': 'LDPEPRVSUSCOUNTY12057'
}

In [33]:
# get data
df = pdr.DataReader(list(param_dict.values()), 'fred', '2020-01-01', '2023-05-01').reset_index()
print('Number of rows:', len(df))
print('Number of columns:', len(df.columns))
df.head()

Number of rows: 40
Number of columns: 4


,DATE,FLHILL7URN,MEDDAYONMAR12057,LDPEPRVSUSCOUNTY12057
0,2020-01-01,3.1,70.0,1.326805
1,2020-02-01,2.7,58.0,1.334155
2,2020-03-01,4.3,50.0,1.326460
3,2020-04-01,12.0,53.0,1.486079
4,2020-05-01,12.9,64.0,1.395393


In [34]:
key_list = []
for selected_v in list(param_dict.values()):
  for k, v in param_dict.items():
    if v == selected_v:
      key_list.append(k)
new_col_list = ['date'] + key_list
df.columns = new_col_list

In [35]:
df.head()

,date,unemployment,median_days_on_market,market_hotness_ratio
0,2020-01-01,3.1,70.0,1.326805
1,2020-02-01,2.7,58.0,1.334155
2,2020-03-01,4.3,50.0,1.326460
3,2020-04-01,12.0,53.0,1.486079
4,2020-05-01,12.9,64.0,1.395393


## <font color="blue">Open AI</font>


In [19]:
# create pandas agent
# full docs - https://python.langchain.com/en/latest/modules/agents/toolkits/examples/pandas.html
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=300), df, verbose=True)

### Question #1 - How has unemployment changed in the last two years?

In [21]:
agent.run('how has unemployment changed from jan 2020 to jan 2023')



> Entering new AgentExecutor chain...
Thought: I need to compare the unemployment rate from jan 2020 to jan 2023
Action: python_repl_ast
Action Input: df[df['date'] == '2023-01-01 00:00:00']['unemployment']
Observation: 36    2.5
Name: unemployment, dtype: float64
Thought: I now need to compare this to the unemployment rate from jan 2020
Action: python_repl_ast
Action Input: df[df['date'] == '2020-01-01 00:00:00']['unemployment']
Observation: 0    3.1
Name: unemployment, dtype: float64
Thought: I now know the final answer
Final Answer: The unemployment rate has decreased from 3.1 in Jan 2020 to 2.5 in Jan 2023.

> Finished chain.


'The unemployment rate has decreased from 3.1 in Jan 2020 to 2.5 in Jan 2023.'

### Question #2 - What is the trend in median days on market for 2023?

In [22]:
agent.run('What is the trend in median days on market for 2023')



> Entering new AgentExecutor chain...
Thought: I need to find the median days on market for 2023
Action: python_repl_ast
Action Input: df[df['date'].str.contains('2023')]['median_days_on_market'].median()
Observation: AttributeError('Can only use .str accessor with string values!')
Thought: I need to convert the date column to a string
Action: python_repl_ast
Action Input: df['date'] = df['date'].astype(str)
Observation: 
Thought: I need to find the median days on market for 2023
Action: python_repl_ast
Action Input: df[df['date'].str.contains('2023')]['median_days_on_market'].median()
Observation: 58.0
Thought: I now know the final answer
Final Answer: The median days on market for 2023 is 58.0.

> Finished chain.


'The median days on market for 2023 is 58.0.'

### Question #3 - How has market hotness changed over the last two months?

In [23]:
agent.run('how has market hotness changed over the last two months')



> Entering new AgentExecutor chain...
Thought: I need to compare the market hotness ratio of the last two months
Action: python_repl_ast
Action Input: df.tail(2)['market_hotness_ratio']
Observation: 38    1.179287
39    1.219342
Name: market_hotness_ratio, dtype: float64
Thought: I now know the final answer
Final Answer: The market hotness ratio has increased by 0.04

> Finished chain.


'The market hotness ratio has increased by 0.04'

## <font color="blue">Want to get started in Real Estate Data Analytics?</font>
Checkout my course below 👇

[Introduction to Real Estate Data Analytics](https://analyticsariel.teachable.com/p/real-estate-data-analytics)


# End Notebook